In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# define some variables to be used throughout the notebook
db2 = "fcsq"

In [ ]:
#Orders made - adding a country variable
print("aggregating order level dataset....")
pydb.create_temp_table( 
f"""
SELECT
  Year,
   Order_desc,
   order_code,  
   CASE WHEN disposal_region = 'WALES' 
         THEN 'Wales'
         ELSE 'England' END 
      AS country,
   COUNT(*) AS Count
FROM
  {db2}.ca_disps_all_children
WHERE
  year BETWEEN 2011 AND 2021  
  AND disp_type_code = 1
  AND order_case_type = 'C'
  AND order_code IN (27,30)
GROUP BY
  Year,
  Order_desc,
  order_code,
  CASE WHEN disposal_region = 'WALES' 
         THEN 'Wales'
         ELSE 'England' END
  
""",

"pub_sgo_cao")

In [ ]:
pub_data = pydb.read_sql_query("SELECT * FROM __temp__.pub_sgo_cao")

In [ ]:
pub_df = pd.DataFrame(pub_data)

In [ ]:
pub_df.to_csv (r's3://alpha-family-data/Adhoc/pub_child_sgo_cao_county.csv', header = True, index=False)